![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

# Custom data history

In [1]:
// We need to load assemblies at the start in their own cell
#load "./Initialize.csx"

In [8]:
// Initialize Lean Engine.
#load "./QuantConnect.csx"

using System.Globalization;
using System.Linq;
using QuantConnect;
using QuantConnect.Data;
using QuantConnect.Algorithm;
using QuantConnect.Research;

In [9]:
class CustomDataType : DynamicData
{
    public decimal Open;
    public decimal High;
    public decimal Low;
    public decimal Close;

    public override SubscriptionDataSource GetSource(SubscriptionDataConfig config, DateTime date, bool isLiveMode)
    {
        var source = "https://www.dl.dropboxusercontent.com/s/d83xvd7mm9fzpk0/path_to_my_csv_data.csv?dl=0";
        return new SubscriptionDataSource(source, SubscriptionTransportMedium.RemoteFile);
    }

    public override BaseData Reader(SubscriptionDataConfig config, string line, DateTime date, bool isLiveMode)
    {
        if (string.IsNullOrWhiteSpace(line.Trim()))
        {
            return null;
        }

        try
        {
            var csv = line.Split(",");
            var data = new CustomDataType()
            {
                Symbol = config.Symbol,
                Time = DateTime.ParseExact(csv[0], DateFormat.DB, CultureInfo.InvariantCulture).AddHours(20),
                Value = csv[4].ToDecimal(),
                Open = csv[1].ToDecimal(),
                High = csv[2].ToDecimal(),
                Low = csv[3].ToDecimal(),
                Close = csv[4].ToDecimal()
            };

            return data;
        }
        catch
        {
            return null;
        }
    }
}

In [11]:
var qb = new QuantBook();
var symbol = qb.AddData<CustomDataType>("CustomDataType", Resolution.Hour).Symbol;

var start = new DateTime(2017, 8, 20);
var end = start.AddHours(48);
var history = qb.History<CustomDataType>(symbol, start, end, Resolution.Hour).ToList();

if (history.Count == 0)
{
    throw new Exception("No history data returned");
}